### TINYGRAD TESTS

In [1]:
from tinygrad import Device
print(Device.DEFAULT)

NV


In [1]:
from tinygrad import Tensor, nn

class Model:
  def __init__(self):
    self.l1 = nn.Conv2d(1, 32, kernel_size=(3,3))
    self.l2 = nn.Conv2d(32, 64, kernel_size=(3,3))
    self.l3 = nn.Linear(1600, 10)

  def __call__(self, x:Tensor) -> Tensor:
    x = self.l1(x).relu().max_pool2d((2,2))
    x = self.l2(x).relu().max_pool2d((2,2))
    return self.l3(x.flatten(1).dropout(0.5))

In [2]:
from tinygrad.nn.datasets import mnist
X_train, Y_train, X_test, Y_test = mnist()
print(X_train.shape, X_train.dtype, Y_train.shape, Y_train.dtype)
# (60000, 1, 28, 28) dtypes.uchar (60000,) dtypes.uchar

https://storage.googleapis.com/cvdf-datasets/mnist/train-images-idx3-ubyte.gz: 4
https://storage.googleapis.com/cvdf-datasets/mnist/train-labels-idx1-ubyte.gz: 6
https://storage.googleapis.com/cvdf-datasets/mnist/t10k-images-idx3-ubyte.gz: 7.
https://storage.googleapis.com/cvdf-datasets/mnist/t10k-labels-idx1-ubyte.gz: 10


(60000, 1, 28, 28) dtypes.uchar (60000,) dtypes.uchar


In [3]:
model = Model()
acc = (model(X_test).argmax(axis=1) == Y_test).mean()
# NOTE: tinygrad is lazy, and hasn't actually run anything by this point
print(acc.item())  # ~10% accuracy, as expected from a random model

0.09049999713897705


In [4]:
optim = nn.optim.Adam(nn.state.get_parameters(model))
batch_size = 128
def step():
  Tensor.training = True  # makes dropout work
  samples = Tensor.randint(batch_size, high=X_train.shape[0])
  X, Y = X_train[samples], Y_train[samples]
  optim.zero_grad()
  loss = model(X).sparse_categorical_crossentropy(Y).backward()
  optim.step()
  return loss

In [5]:
import timeit
timeit.repeat(step, repeat=5, number=1)
#[0.08268719699981375,
# 0.07478952900009972,
# 0.07714716600003158,
# 0.07785399599970333,
# 0.07605237000007037]

[6.0677060359998904,
 0.988821069000096,
 0.06392392399993696,
 0.0623034740001458,
 0.06252255599974887]

In [6]:
from tinygrad import GlobalCounters, Context
GlobalCounters.reset()
with Context(DEBUG=2): step()

scheduled 46 kernels
*** GPU        1 E_n12                                     arg  1 mem  0.06 GB tm      4.10us/     0.00ms (     0.00 GFLOPS    0.0|0.0     GB/s) ['__imul__']
*** GPU        2 E_n13                                     arg  1 mem  0.06 GB tm      5.12us/     0.01ms (     0.00 GFLOPS    0.0|0.0     GB/s) ['__imul__']
*** GPU        3 E_n7                                      arg  1 mem  0.06 GB tm      5.12us/     0.01ms (     0.00 GFLOPS    0.0|0.0     GB/s) ['randint']
*** GPU        4 E_                                        arg  1 mem  0.06 GB tm      5.12us/     0.02ms (     0.00 GFLOPS    0.0|0.0     GB/s) ['randint']
*** GPU        5 r_625_32_15000_3_4                        arg  1 mem  0.06 GB tm      5.12us/     0.02ms (    89.84 GFLOPS   46.9|46.9    GB/s) ['__getitem__']
*** GPU        6 r_5_2_10n1                                arg  1 mem  0.06 GB tm      5.12us/     0.03ms (     0.07 GFLOPS    0.0|0.0     GB/s) ['sparse_categorical_crossentropy']
*** GPU

In [7]:
from tinygrad import TinyJit
jit_step = TinyJit(step)

In [8]:
import timeit
timeit.repeat(jit_step, repeat=5, number=1)
# [0.2596786549997887,
#  0.08989566299987928,
#  0.0012115650001760514,
#  0.001010227999813651,
#  0.0012164899999334011]

[0.32148898800005554,
 0.06408027600036803,
 0.0008017889999791805,
 0.0006849289998172026,
 0.0006680179999420943]

In [10]:
for step in range(700):
  loss = jit_step()
  if step%100 == 0:
    Tensor.training = False
    acc = (model(X_test).argmax(axis=1) == Y_test).mean().item()
    print(f"step {step:4d}, loss {loss.item():.2f}, acc {acc*100.:.2f}%")

step    0, loss 0.09, acc 98.49%
step  100, loss 0.11, acc 98.68%
step  200, loss 0.04, acc 98.59%
step  300, loss 0.09, acc 98.61%
step  400, loss 0.15, acc 98.57%
step  500, loss 0.04, acc 98.49%
step  600, loss 0.12, acc 98.71%


### tinygrad conversation.py debugging ###

In [23]:
from phonemizer.backend import EspeakBackend
from phonemizer.punctuation import Punctuation
from phonemizer.separator import Separator

text = "Hello, World! Preprocess deez nuts!"
text = "No, I'm just text-based, I don't have the ability to hear or see you."
text = "Hello, I'm a chat bot"

# remove all the punctuation from the text, considering only the specified
# punctuation marks
#text = Punctuation(';:,.!"?()').remove(text)

# build the set of all the words in the text
#words = {w.lower() for line in text for w in line.strip().split(' ') if w}
words = [word.lower() for word in text.strip().split(' ')]

# initialize the espeak backend for English
backend = EspeakBackend('en-us')

# separate phones by a space and ignoring words boundaries
separator = Separator(phone=' ', word=None)

# build the lexicon by phonemizing each word one by one. The backend.phonemize
# function expect a list as input and outputs a list.
lexicon = {
    word: backend.phonemize([word], separator=separator, strip=True)[0]
    for word in words}

print(lexicon)

{'hello,': 'h ə l oʊ', "i'm": 'aɪ m', 'a': 'eɪ', 'chat': 'tʃ æ t', 'bot': 'b ɑː t'}


In [24]:
import eng_to_ipa as ipa
ipa.convert(words)

'hɛˈloʊ, əm ə ʧæt bot*'